In [1]:
import sys
sys.path.append('../../Common/')

import CommonMT5

# Chien luoc
Chiến lược em dự kiến xây dựng sẽ là sự kết hợp của nhiều chỉ báo và thuật toán.
Ví dụ: khi giá chọc thủng bollinger band trên —> 1 điểm trend
Khi giá tăng 3 nến xanh liên tiếp —> 1 điểm trend
Khi william rsi vượt 70 —> 1 điểm trend

Theo hướng ngược lại sẽ là -1 điểm

Tổng điểm trend vượt 0 thì sẽ Buy
<= 0 thì sẽ Sell đóng lệnh Long
Ngược lại, điểm trend giảm dưới 0 sẽ short, và >= 0 thì Cover đóng Short

In [2]:
def calculate_bollinger_bands(data, window=20, nb_dev=2):
    """Tính Bollinger Bands mà không cần ta-lib."""
    middle_band = data['Close'].rolling(window=window).mean()
    std_dev = data['Close'].rolling(window=window).std()
    upper_band = middle_band + (std_dev * nb_dev)
    lower_band = middle_band - (std_dev * nb_dev)
    return upper_band, middle_band, lower_band

def calculate_williams_r(data, period=14):
    """Tính Williams %R mà không cần ta-lib."""
    high = data['High'].rolling(window=period).max()
    low = data['Low'].rolling(window=period).min()
    will_r = -100 * ((high - data['Close']) / (high - low))
    return will_r

def calculate_trend(data):
    # Tính Bollinger Bands
    upperband, middleband, lowerband = calculate_bollinger_bands(data)
    
    # Tính William RSI
    will_r = calculate_williams_r(data)
    
    # Điểm trend
    data['trend_point'] = 0
    
    # Điểm cho Bollinger Bands
    data.loc[data['Close'] > upperband, 'trend_point'] += 1
    data.loc[data['Close'] < lowerband, 'trend_point'] -= 1
    
    # Điểm cho 3 nến xanh liên tiếp
    green_candle = data['Close'] > data['Open']
    data['trend_point'] += ((green_candle & green_candle.shift(1) & green_candle.shift(2)).astype(int))
    
    # Điểm cho William RSI
    data.loc[will_r > -20, 'trend_point'] += 1  # William RSI trả về giá trị âm, -20 tương đương với 80 trên thang điểm thông thường
    data.loc[will_r < -80, 'trend_point'] -= 1
    
    # Áp dụng hàm tín hiệu mua và bán
    data['Buy_Signal'] = data.apply(lambda row: buy_signal(row), axis=1)
    data['Sell_Signal'] = data.apply(lambda row: sell_signal(row), axis=1)

    return data

def buy_signal(row):
    """Định nghĩa tín hiệu mua."""
    return row['trend_point'] > 0
    # return True

def sell_signal(row):
    """Định nghĩa tín hiệu bán."""
    return row['trend_point'] < 0
    # return True

def detectSignal(symbol, from_date, to_date, timeframe):

    import pandas as pd
    import plotly.graph_objects as go
    import redis
    import numpy as np
    import ta
    from datetime import datetime

    # ##############################################Step 1: Lấy dữ liệu##############################################
    data = CommonMT5.CommonMT5.loaddataMT5_FromTo(symbol, from_date, to_date, timeframe)
    
    # ##############################################Step 2: Chiến lược##############################################  
    # Tính toán các chỉ báo kỹ thuật
    
    data = calculate_trend(data)

    # Áp dụng hàm tín hiệu mua và bán
    data['Buy_Signal'] = data.apply(lambda row: buy_signal(row), axis=1)
    data['Sell_Signal'] = data.apply(lambda row: sell_signal(row), axis=1)
    
    # ##############################################Step 3: Đẩy dữ liệu qua Redis##############################################
    # Nếu có tín hiệu thì đẩy qua Redis
    # Datetime  Open    High    Low	Close   Volume  SMA short_ema   long_ema    MACD    Signal_Line Buy_Signal  Sell_Signal
    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)
    # Tạo hash key
    hash_key = 'OG_KetHop'
    last_record = data.iloc[-1]
   
    # Chuyển đổi record cuối cùng thành từ điển và lưu vào Redis dưới dạng hash
    if(last_record['Buy_Signal'] == True or last_record['Sell_Signal'] == True):
        for field, value in last_record.to_dict().items():
            # Chuyển đổi giá trị uint64 và Timestamp thành chuỗi
            if isinstance(value, pd.Timestamp):
                value = value.isoformat()
            elif isinstance(value, (int, np.uint64)):
                value = str(value)
            r.hset(hash_key, field, value)  
            r.hset(hash_key, 'Symbol', symbol)
            r.hset(hash_key, 'Insertdate', datetime.now().isoformat())
        print(last_record)   
    else:
        print('Không có tín hiệu!')

In [3]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import ta
import schedule
import time

def schedule_job():
    symbol = 'EURUSD'
    from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    timeframe = mt5.TIMEFRAME_M15
    detectSignal(symbol, from_date, to_date, timeframe)

# Lên lịch chạy hàm tại các phút cụ thể trong mỗi giờ
for hour in range(24):  # Loop from 0 to 23
    for minute in [1, 16, 31, 49]:  # Các phút cụ thể
        # Format the time string with leading zeros for hours and minutes
        time_string = "{:02d}:{:02d}".format(hour, minute)
        schedule.every().day.at(time_string).do(schedule_job)

while True:
    schedule.run_pending()
    time.sleep(1)

Datetime       2024-02-26 02:45:00
Open                       1.08184
High                       1.08194
Low                        1.08178
Close                      1.08179
Volume                         158
trend_point                     -1
Buy_Signal                    True
Sell_Signal                   True
Name: 327, dtype: object
